In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
import pandas as pd
import datetime as dt
import plotly

In [ ]:
# select json to generate data from
OD_FILE = "fire_greece_ps.json"
VAR_FILE = ""

In [ ]:


with open(OD_FILE) as f:
    data = json.load(f)

# generate variables to be added to the df
vidlength = data['videoInfo']['videoLength']
unique_keys = set([_key for key,val in data['seconds'].items() for _key, _val in val.items()])
df = pd.DataFrame({'seconds': range(0, vidlength)})
df['seconds'] = pd.to_datetime(df["seconds"], unit='s').dt.strftime('%H:%M:%S')

#add placeholder values
for unique_key in unique_keys:
    df[unique_key] = 0

#impute real values in their corresponding seconds
for second, vals in data['seconds'].items():
    for _key, _val in vals.items():

        df.at[int(second), _key] = _val

# reorder columns
df = df[['seconds', 'Officer', 'Civilian', 'Chemical Smoke', 'Riot Shield', 'Baton']]

In [ ]:
df

In [ ]:

#keep track of detection colors
detection_colors = {'Officer': 'blue', 'Civilian': 'blue', 'Baton': 'green', 'Gun':'green', 'Riot Shield': 'green', 'Chemical Smoke': 'red'}

# make the subplots with corresponding universal values
fig = make_subplots(
    rows=len(df.columns)-1, 
    cols=1,
    #shared_xaxes=True,
    subplot_titles=df.columns[1:],
    vertical_spacing=0,
    column_widths=[1],
    )

# add the bars into the graph
for i, val in enumerate(df.columns):
    if val == 'seconds':
        continue
    fig.append_trace(
        go.Bar(
        x=df['seconds'],
        y=df[val],
        marker=dict(color=detection_colors[val])
    ),row=i, col=1),
    # add negative of same values to flip the bars on the same graph
    fig.append_trace(
        go.Bar(
        x=df['seconds'],
        y=df[val] * -1,
        marker=dict(color=detection_colors[val])
    ),row=i, col=1)

# adjust universal values surroinding the graph, and bar types
fig.update_layout(
    showlegend=False,
    barmode='relative',
    height = 400,
    width = 1500,
    margin=dict(
    l=150,
    r=0,
    b=0,
    t=30,
    pad=0
    ),
    )

# update axes of graphs
for i in range(1, len(df.columns)):
    fig.update_yaxes(
    row=i, col=1, range=[-10,10], side='left', showticklabels=False, showgrid=False, linewidth=1, linecolor='black', mirror=True,)

    if i == 1:
        fig.update_xaxes(row=i, col=1, side='top',linewidth=1, linecolor='gray', mirror=True,nticks=12)
    else:
        fig.update_xaxes(row=i, col=1, showticklabels=False, linewidth=1, linecolor='gray', mirror=True)

# go into the dictionary to make extremely custom changes to locations of items in graph
for i in range(0, len(df.columns[1:])):
    fig.layout.annotations[i].update(x=-0.06)
    fig.layout.annotations[i]['y'] = fig.layout.annotations[i]['y'] - 0.13

fig.show()

In [ ]:
# export graph
# 
# plotly.io.write_json(fig, f'{file_name_split}_plotly.json', validate=True, pretty=False, remove_uids=True)


In [ ]:
#function before sending to remove needless data
# create duration list
# remove all instances of time

def format_and_export_plotly_to_json(fig):

    file_name_split = FILE.split('.')[0]
    export_json_name = f'{file_name_split}_plotly.json'

    plotly.io.write_json(fig, export_json_name, validate=True, pretty=False, remove_uids=True)

    with open(export_json_name) as f:
        data = json.load(f)

    time_list = df['seconds'].to_list()

    # ALWAYS append time list to end of data list
    data['data'].append({'duration': time_list})

    for i in data['data']:
        
        # if i['x'] exists
        try:
            del i['x']
        except:
            continue
    
    with open(export_json_name, "w") as outfile:
        json.dump(data, outfile)



format_and_export_plotly_to_json(fig)